In [1]:
import pandas as pd
from datetime import datetime, timezone, timedelta
from matplotlib import pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_feather("data/dfs/complete_data.feather")
screening_start_1 = pd.to_datetime("2021-05-10 00:00")
screening_end_1 = pd.to_datetime("2021-05-16 00:00")
screening_start_2 = pd.to_datetime("2021-05-17 00:00")
screening_end_2 = pd.to_datetime("2021-05-23 00:00")
alias_col = df["TIMESTAMP"]
filtering_alias = alias_col.map(lambda x: (x > screening_start_1 and x < screening_end_1) or (
    x > screening_start_2 and x < screening_end_2))
df = df[filtering_alias]
# df.to_feather()


In [3]:
new_df = df.copy()
localized = new_df["TIMESTAMP"].dt.tz_localize("CET")
new_df["DAY"] = [x.day_name() for x in localized]
print(new_df["DAY"].unique())
thursday_only = new_df[new_df["DAY"] == "Thursday"]
data_band_19 = thursday_only[thursday_only["ALIAS"] == "Band 23"]
data_band_19


['Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday']


,TIMESTAMP,RAW_INTENSITY,STEPS,RAW_KIND,HEART_RATE,ALIAS,Rok narození,Pohlaví,Výška [cm],Váha [kg],Rukovost,Přes noc budu mít náramek na ruce,DAY
1495204,2021-05-13 00:00:00,0.0,0.0,112.0,71.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
1495205,2021-05-13 00:01:00,0.0,0.0,112.0,255.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
1495206,2021-05-13 00:02:00,0.0,0.0,112.0,70.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
1495207,2021-05-13 00:03:00,0.0,0.0,112.0,255.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
1495208,2021-05-13 00:04:00,0.0,0.0,112.0,72.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506719,2021-05-20 23:55:00,45.0,0.0,121.0,255.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
1506720,2021-05-20 23:56:00,0.0,0.0,121.0,71.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
1506721,2021-05-20 23:57:00,0.0,0.0,122.0,255.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday
1506722,2021-05-20 23:58:00,0.0,0.0,112.0,79.0,Band 23,2012,Muž,139,31.0,Levoruký,Ano,Thursday


In [4]:
df_schedule = pd.read_csv("data/schedule.csv")
# df_schedule = df_schedule[df_schedule["day"] == "Thursday"]
# df_schedule

,day,subject,from,to,school
13,Thursday,Czech,8:00,8:45,True
14,Thursday,Math,8:55,9:40,True
15,Thursday,Czech,9:50,10:35,True
16,Thursday,Physical Education,11:00,11:45,True
17,Thursday,Natural Science,11:55,12:40,True


In [42]:
def append_schedule_info(row):
    #     print(type(row))
    #     print(row)
    
    daily_schedule = df_schedule[df_schedule["day"] == row["DAY"]]
    daily_schedule["from"] = pd.to_datetime(daily_schedule["from"], format="%H:%M")
    daily_schedule["to"] = pd.to_datetime(daily_schedule["to"], format="%H:%M")
    current_time = pd.to_datetime(row["TIMESTAMP"])
    print(daily_schedule["to"].dtype)
    print(type(current_time))
    active_lesson = daily_schedule[daily_schedule["from"] <= current_time and current_time < daily_schedule["to"]]

data_band_19["SCHEDULE_INFO"] = data_band_19.apply(append_schedule_info, axis=1)
data_band_19.head()

datetime64[ns]
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [9]:
# plt.scatter(x="TIMESTAMP", y="HEART_RATE", data=data_band_19)
fig = px.scatter(x=data_band_19["TIMESTAMP"], y=data_band_19["STEPS"])
fig.show()